In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from patsy import dmatrices

In [3]:
wine = pd.read_csv("winequality-red.csv")
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
wine.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [5]:
wine.isnull().sum()
type(wine)

pandas.core.frame.DataFrame

## rename을 사용해서 특성명에 존재하는 공백을 ' _ '로 대체하기

In [6]:
wine.rename(columns={'fixed acidity' : 'fixed_acidity', 'volatile acidity': 'volatile_acidity', 
                    'citric acid': 'citric_acid', 'residual sugar': 'residual_sugar', 
                    'free sulfur dioxide': 'free_sulfur_dioxide', 'total sulfur dioxide': 'total_sulfur_dioxide'}, inplace=True)
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         1599 non-null   float64
 1   volatile_acidity      1599 non-null   float64
 2   citric_acid           1599 non-null   float64
 3   residual_sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free_sulfur_dioxide   1599 non-null   float64
 6   total_sulfur_dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


### 각 특성이 quality와 상관관계(?)에서 alcohol, sulphates, citric_acid, fixed_acidity (추가로, residual_sugar)를 제외하고 음의 상관관계를 보임

In [7]:
correlation = wine.corr()
correlation['quality'].sort_values(ascending=False)

quality                 1.000000
alcohol                 0.476166
sulphates               0.251397
citric_acid             0.226373
fixed_acidity           0.124052
residual_sugar          0.013732
free_sulfur_dioxide    -0.050656
pH                     -0.057731
chlorides              -0.128907
density                -0.174919
total_sulfur_dioxide   -0.185100
volatile_acidity       -0.390558
Name: quality, dtype: float64

# Shuffle !!!!!!!!!!!!!!!!!!!!

In [8]:
matrix_shuffle = wine.iloc[np.random.permutation(wine.index)].reset_index(drop=True)

In [9]:
y_shuffle = matrix_shuffle["quality"]
X_shuffle = matrix_shuffle.drop(["quality"], axis=1)

In [10]:
X_train_new = correlation['quality'].sort_values(ascending=False)
X_train_new = X_train_new[1:4]
X_train_new

alcohol        0.476166
sulphates      0.251397
citric_acid    0.226373
Name: quality, dtype: float64

In [11]:
# wine['rate_code'] = (wine['quality'] > 4).astype(np.float32)
# wine['rate_code'] = wine['quality'].copy()

In [12]:
import sklearn
import sklearn.metrics
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

model = sklearn.linear_model.LogisticRegression(max_iter = 5000, random_state=42)
y_train_new, X = dmatrices('quality ~ alcohol + sulphates + citric_acid + fixed_acidity', data = wine)
# model.fit(X, y.ravel())
# cross_val_score(model, X, y.ravel(), cv=10, scoring="accuracy")

In [13]:
np.mean([0.43125   , 0.5375    , 0.5875    , 0.49375   , 0.6       ,
       0.69375   , 0.61875   , 0.49375   , 0.625     , 0.57861635])

0.565986635

### 특성 중 상관계수가 ±0.0대인것들 제외하고 해보자

In [14]:
y_train_new, X_new = dmatrices('quality ~ alcohol + sulphates + citric_acid + fixed_acidity + residual_sugar + chlorides + density + total_sulfur_dioxide + volatile_acidity', data=wine)

In [15]:
X_new

DesignMatrix with shape (1599, 10)
  Columns:
    ['Intercept',
     'alcohol',
     'sulphates',
     'citric_acid',
     'fixed_acidity',
     'residual_sugar',
     'chlorides',
     'density',
     'total_sulfur_dioxide',
     'volatile_acidity']
  Terms:
    'Intercept' (column 0)
    'alcohol' (column 1)
    'sulphates' (column 2)
    'citric_acid' (column 3)
    'fixed_acidity' (column 4)
    'residual_sugar' (column 5)
    'chlorides' (column 6)
    'density' (column 7)
    'total_sulfur_dioxide' (column 8)
    'volatile_acidity' (column 9)
  (to view full data, use np.asarray(this_obj))

In [16]:
y_train_new.shape

(1599, 1)

In [19]:
y_train_new.ravel()

array([5., 5., 5., ..., 6., 5., 6.])

In [20]:
X

DesignMatrix with shape (1599, 5)
  Intercept  alcohol  sulphates  citric_acid  fixed_acidity
          1      9.4       0.56         0.00            7.4
          1      9.8       0.68         0.00            7.8
          1      9.8       0.65         0.04            7.8
          1      9.8       0.58         0.56           11.2
          1      9.4       0.56         0.00            7.4
          1      9.4       0.56         0.00            7.4
          1      9.4       0.46         0.06            7.9
          1     10.0       0.47         0.00            7.3
          1      9.5       0.57         0.02            7.8
          1     10.5       0.80         0.36            7.5
          1      9.2       0.54         0.08            6.7
          1     10.5       0.80         0.36            7.5
          1      9.9       0.52         0.00            5.6
          1      9.1       1.56         0.29            7.8
          1      9.2       0.88         0.18            8.9
      

In [21]:
X_train_new = X[:, 1:]
X_train_new

array([[ 9.4 ,  0.56,  0.  ,  7.4 ],
       [ 9.8 ,  0.68,  0.  ,  7.8 ],
       [ 9.8 ,  0.65,  0.04,  7.8 ],
       ...,
       [11.  ,  0.75,  0.13,  6.3 ],
       [10.2 ,  0.71,  0.12,  5.9 ],
       [11.  ,  0.66,  0.47,  6.  ]])

In [22]:
def display_score(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# 참조
## https://github.com/vikrantkakad/Red-Wine-Quality-Analysis/blob/master/Capstone_Vikrant.Kakad.ipynb

# Self analysis

In [23]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         1599 non-null   float64
 1   volatile_acidity      1599 non-null   float64
 2   citric_acid           1599 non-null   float64
 3   residual_sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free_sulfur_dioxide   1599 non-null   float64
 6   total_sulfur_dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [24]:
data_train = wine.to_numpy()

In [26]:
X_train = data_train[:, :-1]
X_train

array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
       [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
       [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
       ...,
       [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
       [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
       [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]])

In [27]:
y_train = data_train[:, -1]
y_train


array([5., 5., 5., ..., 6., 5., 6.])

# 암것도 안하고 회귀 분석. 이후에 양의 상관계수를 가진 특성들만 가지고 훈련해볼 것

## 1-1. LogisticRegression [solver="liblinear"]

In [29]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression(solver="liblinear", C=10, random_state=42)
log_reg_score = cross_val_score(log_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
log_rmse_score = np.sqrt(-log_reg_score)
display_score(log_rmse_score)

Scores: [0.81009259 0.71589105 0.70267347 0.79843597 0.76239753 0.6846532
 0.66143783 0.75415516 0.72886899 0.72684378]
Mean: 0.7345449564089788
Standard deviation: 0.04498772825312228


## 1-2. LogisticRegression [C=5 (decrease)]

In [22]:
log_reg2 = LogisticRegression(solver="liblinear", random_state=42)
log_reg_score2 = cross_val_score(log_reg2, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
log_rmse_score2 = np.sqrt(-log_reg_score2)
display_score(log_rmse_score2)

Scores: [0.79056942 0.73314391 0.71589105 0.78262379 0.76239753 0.70710678
 0.66614563 0.74582169 0.70710678 0.7004042 ]
Mean: 0.731121079193854
Standard deviation: 0.03733056833212232


## display_score 선언

## 2. SVM Regression

# 2-1. Using kernel SVM Model 이건 오래걸리므로 나중에 하자

In [ ]:
# from sklearn.svm import LinearSVR
# from sklearn.svm import SVR
# svm_poly = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="auto")

# svm_poly_score = cross_val_score(svm_poly, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
# svm_poly_rmse = np.sqrt(-svm_poly_score)
# display_score(svm_poly_rmse)

In [ ]:
# cross_val_score(svm_poly_reg, X_train, y_train, cv=10, scoring="accuracy")

### 2-2. Using Linear SVM Model

#### svm_reg.fit(X, y) 빼고 바로 cross_val_score 돌리면 안대냐? ;;

In [23]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42, max_iter = 10000)
svm_reg_score = cross_val_score(svm_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
svm_rmse_score = np.sqrt(-svm_reg_score)
display_score(svm_rmse_score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

Scores: [0.85553596 0.64885695 0.78267117 0.71928669 0.61822008 0.7446786
 0.63665214 0.65463868 0.61398251 0.72416424]
Mean: 0.6998687034651347
Standard deviation: 0.07533327592039181


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


## Gradient Boosting

In [24]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(random_state=42)
gbrt_score = cross_val_score(gbrt, X_train, y_train, scoring = "neg_mean_squared_error", cv = 10)
gbrt_rmse_score = np.sqrt(-gbrt_score)
display_score(gbrt_rmse_score)

Scores: [0.62545082 0.61762548 0.67620986 0.67963328 0.57786445 0.71376148
 0.67820201 0.64711995 0.6045297  0.7012463 ]
Mean: 0.6521643328204947
Standard deviation: 0.04235027764402308


## Random Forest Regressor

In [30]:
from sklearn.ensemble import RandomForestRegressor
rnd_reg = RandomForestRegressor(random_state=42)
rnd_reg_score = cross_val_score(rnd_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
rnd_rmse_score = np.sqrt(-rnd_reg_score)
display_score(rnd_rmse_score)

Scores: [0.61371614 0.58156631 0.68472075 0.66384722 0.5649209  0.72317443
 0.66583876 0.65855239 0.61977667 0.70264281]
Mean: 0.6478756369545334
Standard deviation: 0.04894232181514159


# Decision Tree Regressor

In [31]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth = 2, random_state = 42)
tree_reg_score = cross_val_score(tree_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
tree_rmse_score = np.sqrt(-tree_reg_score)
display_score(tree_rmse_score)

Scores: [0.73529954 0.71168031 0.79269341 0.76499069 0.63282562 0.79991351
 0.74242189 0.74317426 0.66748292 0.76945147]
Mean: 0.7359933619216152
Standard deviation: 0.05028193557207711


# Bagging Regressor (using R-F Regressor)

In [ ]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor(
    RandomForestRegressor(random_state=42), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1, random_state = 42)
bag_score = cross_val_score(bag_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
bag_rmse_score = np.sqrt(-bag_score)
display_score(bag_rmse_score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py", line 244, in fit
    return self._fit(X, y, self.max_samples, sample_weight=sample_weight)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py", line 370, in _fit
    all_results = Parallel(n_jobs=n_jobs, verbose=self.verbose,
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 1054, in __call__
    self.retrieve()
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 933, in retrieve
    self._output.extend(job.get(

# Voting Regressor

In [31]:
# from sklearn.ensemble import VotingRegressor
# from sklearn.svm import SVC

# log_reg = LogisticRegression(solver="liblinear", random_state=42)
# rnd_reg = RandomForestRegressor(random_state=42)
# svm_reg = LinearSVR(epsilon=1.5, random_state=42)

# voting_reg = VotingRegressor(
#     estimators=[('lr', log_reg), ('rf', rnd_reg), ('svc', svm_reg)])

# voting_score = cross_val_score(voting_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
# voting_rmse_score = np.sqrt(-voting_score)
# display_score(voting_rmse_score)

Scores: [nan nan nan nan nan nan nan nan nan nan]
Mean: nan
Standard deviation: nan


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_voting.py", line 485, in fit
    return super().fit(X, y, sample_weight)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_voting.py", line 66, in fit
    names, clfs = self._validate_estimators()
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_base.py", line 241, in _validate_estimators
    raise ValueError(
ValueError: The estimator LogisticRegression should be a regressor.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\ProgramData\Anaco

## 3. AdaBoost는 개별 분류기 다 돌려보고 성능 좋은걸로 돌립시다

In [38]:
from sklearn.ensemble import AdaBoostRegressor
ada_reg = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=5), n_estimators=1000,
    learning_rate = 0.01, random_state=42)
ada_reg_score = cross_val_score(ada_reg, X_train, y_train, scoring = "neg_mean_squared_error", cv=10)
ada_rmse_score = np.sqrt(-ada_reg_score)
display_score(ada_rmse_score)

Scores: [0.64066127 0.6053048  0.67202026 0.64597068 0.53605835 0.71734926
 0.68753033 0.66042165 0.59559969 0.69106477]
Mean: 0.64519810637125
Standard deviation: 0.05102635304230313


# 이제부터 음의 상관계수를 띄는 특성들을 제거하고 훈련

In [28]:
log_reg = LogisticRegression(solver="liblinear", C=10, random_state=42)
log_reg_score = cross_val_score(log_reg, X_train_new, y_train, scoring = "neg_mean_squared_error", cv=10)
log_rmse_score = np.sqrt(-log_reg_score)
display_score(log_rmse_score)

NameError: name 'LogisticRegression' is not defined

In [83]:
gbrt = GradientBoostingRegressor(random_state=42)
gbrt_score = cross_val_score(gbrt, X_train_new, y_train, scoring = "neg_mean_squared_error", cv = 10)
gbrt_rmse_score = np.sqrt(-gbrt_score)
display_score(gbrt_rmse_score)

Scores: [0.68709093 0.63585793 0.67414365 0.68678506 0.57852584 0.75033954
 0.65457202 0.70013245 0.62911238 0.73891868]
Mean: 0.6735478480508899
Standard deviation: 0.04901056106841354


## GitHub 참조 코드

In [85]:
import sklearn
import sklearn.metrics
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

log_reg = sklearn.linear_model.LogisticRegression(max_iter = 5000, random_state=42)
y, X = dmatrices('quality ~ alcohol + sulphates + citric_acid + fixed_acidity', data = wine)
# model.fit(X, y.ravel())
# cross_val_score(model, X, y.ravel(), cv=10, scoring="accuracy")
log_score = cross_val_score(log_reg, X, y.ravel(), scoring = "neg_mean_squared_error", cv = 10)
log_rmse_score = np.sqrt(-log_score)
display_score(log_rmse_score)

Scores: [0.8253787  0.77862058 0.72024301 0.85146932 0.73739406 0.6846532
 0.68920244 0.77459667 0.698212   0.76889204]
Mean: 0.7528662014620664
Standard deviation: 0.0541939069584882


# Shuffle 이후 다시 해봅시다
### X_train => X_shuffle // y_train -> y_shuffle

In [90]:
log_reg = LogisticRegression(solver="liblinear", C=10, random_state=42)
log_reg_score = cross_val_score(log_reg, X_shuffle, y_shuffle, scoring = "neg_mean_squared_error", cv=10)
log_rmse_score = np.sqrt(-log_reg_score)
display_score(log_rmse_score)

Scores: [0.63245553 0.75       0.74161985 0.77862058 0.75828754 0.6846532
 0.75       0.70267347 0.72456884 0.72250438]
Mean: 0.724538338023898
Standard deviation: 0.04025487045318829


In [91]:
gbrt = GradientBoostingRegressor(random_state=42)
gbrt_score = cross_val_score(gbrt, X_shuffle, y_shuffle, scoring = "neg_mean_squared_error", cv = 10)
gbrt_rmse_score = np.sqrt(-gbrt_score)
display_score(gbrt_rmse_score)

Scores: [0.63917241 0.60097211 0.622932   0.56342159 0.60216987 0.61328411
 0.62342671 0.68386438 0.61197319 0.60022336]
Mean: 0.616143972670955
Standard deviation: 0.029495132856633105


In [32]:
rnd_reg = RandomForestRegressor(n_estimators = 100, random_state=42)
rnd_reg_score = cross_val_score(rnd_reg, X_shuffle, y_shuffle, scoring = "neg_mean_squared_error", cv=10)
rnd_rmse_score = np.sqrt(-rnd_reg_score)
display_score(rnd_rmse_score)

Scores: [0.48413841 0.55282965 0.51164868 0.52362439 0.56412709 0.62620683
 0.60666455 0.56633747 0.61227343 0.57928714]
Mean: 0.5627137637456441
Standard deviation: 0.04372231789118146


In [97]:
rnd_reg = RandomForestRegressor(n_estimators = 1000, oob_score=True, random_state=42)
rnd_reg_score = cross_val_score(rnd_reg, X_shuffle, y_shuffle, scoring = "neg_mean_squared_error", cv=10)
rnd_rmse_score = np.sqrt(-rnd_reg_score)
display_score(rnd_rmse_score)

Scores: [0.62271523 0.52321497 0.55727388 0.51158306 0.55095288 0.56611497
 0.58607875 0.68311766 0.53490312 0.56825423]
Mean: 0.5704208752831044
Standard deviation: 0.048173968164455176


## Using GradientBoosting. less than R-F Regressor

In [99]:
gdb_reg = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=3, max_features='sqrt',
                                    min_samples_leaf=15, min_samples_split=10, loss='huber')
gdb_score = cross_val_score(gdb_reg, X_shuffle, y_shuffle, scoring = "neg_mean_squared_error", cv=10)
gdb_rmse_score = np.sqrt(-gdb_score)
display_score(gdb_rmse_score)

Scores: [0.66190794 0.58670547 0.59029032 0.5577623  0.62219569 0.61504266
 0.64881921 0.71083583 0.57486387 0.58347325]
Mean: 0.6151896533754406
Standard deviation: 0.044578211487857386


## Using AdaBoost. 최저값!

In [100]:
from sklearn.ensemble import AdaBoostRegressor
ada_reg = AdaBoostRegressor(
    RandomForestRegressor(random_state=42), n_estimators=1000,
    learning_rate = 0.01, random_state=42)
ada_reg_score = cross_val_score(ada_reg, X_shuffle, y_shuffle, scoring = "neg_mean_squared_error", cv=10)
ada_rmse_score = np.sqrt(-ada_reg_score)
display_score(ada_rmse_score)

Scores: [0.61119811 0.5157646  0.5327341  0.50223376 0.54339327 0.55719891
 0.57113921 0.67735192 0.5249994  0.56001628]
Mean: 0.5596029578675488
Standard deviation: 0.04905204832890824


## AdaBosst가 0.57 이상 나온다면...?

In [105]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    train_errors, val_errors =[], []
    for m in range(1, len(X_train)):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
        
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")